#### Installed required packages

In [ ]:
# pip install pymongo


#### Created pymongo connection string

In [1]:
import os
import pandas as pd
pd.set_option("display.max_column", None)

from pymongo import MongoClient
from bson import json_util
import json

mongo_url = input("Enter the Mongo connection string: ")
client = MongoClient(mongo_url)
print("Python-MongoDB connection created succesfully -\n", client)


Python-MongoDB connection created succesfully -
 MongoClient(host=['3.146.176.180:27017'], document_class=dict, tz_aware=False, connect=True)


#### List of collections present in the database

In [2]:
db = client['talentsync-backend-production']
collection_names = db.list_collection_names()
print(collection_names)


['candidate_questions', 'job_hr_questions', 'candidates', 'auth_keys', 'jobs', 'audit_logs', 'job_hr_settings', 'batches', 'questions', 'candidates_errors', 'users', 'companies', 'messages', 'auth_tokens']


#### Finding out all the documents present in the 'batches' collection

In [3]:
collection = db['candidate_questions']
results = collection.find()
print(results)


In [4]:
collection = db['candidate_questions']
results = collection.find()
for doc in results:
    print(doc, end='\n\n')


{'_id': ObjectId('6821f0a86342bdaa1172396c'), 'candidate_id': '6821f0883d9e6a1581409215', 'questions': ['Given your extensive experience in human resources, can you describe a project where you utilized Python to automate HR processes or data analysis? What was the outcome?', 'How have you applied your understanding of object-oriented programming principles in any programming projects, and can you provide a specific example of how this improved your code quality?', "In your previous roles, how did you collaborate with technical teams to implement software solutions, and can you share an instance where your input significantly influenced the project's direction?", 'Can you discuss a time when you had to debug a complex issue in a software application? What approach did you take, and what tools did you use to resolve it?', 'As you transition from HR to a technical role, what steps have you taken to enhance your Python skills and understanding of frameworks like Django or Flask?'], 'creat

#### View the very first document of the collection

In [6]:
collection = db['candidate_questions']
results = collection.find_one()
print(results)


{'_id': ObjectId('6821f0a86342bdaa1172396c'), 'candidate_id': '6821f0883d9e6a1581409215', 'questions': ['Given your extensive experience in human resources, can you describe a project where you utilized Python to automate HR processes or data analysis? What was the outcome?', 'How have you applied your understanding of object-oriented programming principles in any programming projects, and can you provide a specific example of how this improved your code quality?', "In your previous roles, how did you collaborate with technical teams to implement software solutions, and can you share an instance where your input significantly influenced the project's direction?", 'Can you discuss a time when you had to debug a complex issue in a software application? What approach did you take, and what tools did you use to resolve it?', 'As you transition from HR to a technical role, what steps have you taken to enhance your Python skills and understanding of frameworks like Django or Flask?'], 'creat

In [9]:
collection = db['candidate_questions']
results = collection.find_one()
print(json.dumps(results, indent=4, default=json_util.default))


{
    "_id": {
        "$oid": "6821f0a86342bdaa1172396c"
    },
    "candidate_id": "6821f0883d9e6a1581409215",
    "questions": [
        "Given your extensive experience in human resources, can you describe a project where you utilized Python to automate HR processes or data analysis? What was the outcome?",
        "How have you applied your understanding of object-oriented programming principles in any programming projects, and can you provide a specific example of how this improved your code quality?",
        "In your previous roles, how did you collaborate with technical teams to implement software solutions, and can you share an instance where your input significantly influenced the project's direction?",
        "Can you discuss a time when you had to debug a complex issue in a software application? What approach did you take, and what tools did you use to resolve it?",
        "As you transition from HR to a technical role, what steps have you taken to enhance your Python sk

#### Finding out the number of documents present in the collection 

In [10]:
# print("shape: ", df.shape)


In [11]:
num_rows = collection.count_documents({})
print("shape:", num_rows )


shape: 386


#### Finding out the unique fields present in this collection

In [ ]:
# print("columns: ", df.columns)


In [13]:
all_fields = set()
for field in collection.find():
    all_fields.update(field.keys())
    
print("Columns:", list(all_fields))


Columns: ['questions', '_id', 'candidate_id', 'updated_at', 'created_at']


#### Finding out a info about the each field present in the collection


In [14]:
from collections import defaultdict
import pprint


field_info = defaultdict(lambda: {"count": 0, "types": set()})
total_docs = 0

# Iterate over all documents in the collection
for doc in collection.find({}):
    total_docs += 1
    for field, value in doc.items():
        field_info[field]["count"] += 1
        field_info[field]["types"].add(type(value).__name__)

# Display like df.info()
print(f"{'Field':<25} {'Non-Null Count':<15} {'Data Types'}")
print("-" * 60)
for field, info in field_info.items():
    print(f"{field:<25} {info['count']:<15} {', '.join(sorted(info['types']))}")

print(f"\nTotal Documents: {total_docs}")


Field                     Non-Null Count  Data Types
------------------------------------------------------------
_id                       386             ObjectId
candidate_id              386             ObjectId, str
questions                 386             list
created_at                386             datetime
updated_at                386             datetime

Total Documents: 386


#### Find what are the chunks of questions are there for every candidates

In [ ]:
# df[['questions']]


In [16]:
query = [
    {
        "$project": {
            "_id": 0,
            "questions": 1
        }
    }
]

result = list(collection.aggregate(query))

for doc in result:
    print(doc, end='\n\n')


{'questions': ['Given your extensive experience in human resources, can you describe a project where you utilized Python to automate HR processes or data analysis? What was the outcome?', 'How have you applied your understanding of object-oriented programming principles in any programming projects, and can you provide a specific example of how this improved your code quality?', "In your previous roles, how did you collaborate with technical teams to implement software solutions, and can you share an instance where your input significantly influenced the project's direction?", 'Can you discuss a time when you had to debug a complex issue in a software application? What approach did you take, and what tools did you use to resolve it?', 'As you transition from HR to a technical role, what steps have you taken to enhance your Python skills and understanding of frameworks like Django or Flask?']}

{'questions': ['Can you describe any experience you have with Python, particularly in data ana

#### Find only the first question in the collection

In [17]:
doc = collection.find(
    {},  # No filter, match all documents
    {"_id": 0, "questions": 1}  # Project only 'questions' field
).limit(1)

# Get the first document
first_question_doc = list(doc)[0]
print(first_question_doc['questions'])


['Given your extensive experience in human resources, can you describe a project where you utilized Python to automate HR processes or data analysis? What was the outcome?', 'How have you applied your understanding of object-oriented programming principles in any programming projects, and can you provide a specific example of how this improved your code quality?', "In your previous roles, how did you collaborate with technical teams to implement software solutions, and can you share an instance where your input significantly influenced the project's direction?", 'Can you discuss a time when you had to debug a complex issue in a software application? What approach did you take, and what tools did you use to resolve it?', 'As you transition from HR to a technical role, what steps have you taken to enhance your Python skills and understanding of frameworks like Django or Flask?']


In [18]:
query = [
    {
        "$project": {
            "_id": 0,
            "questions": 1
        }
    },
    {
        "$limit": 1
    }
]

result = list(collection.aggregate(query))
print(result[0]['questions'])


['Given your extensive experience in human resources, can you describe a project where you utilized Python to automate HR processes or data analysis? What was the outcome?', 'How have you applied your understanding of object-oriented programming principles in any programming projects, and can you provide a specific example of how this improved your code quality?', "In your previous roles, how did you collaborate with technical teams to implement software solutions, and can you share an instance where your input significantly influenced the project's direction?", 'Can you discuss a time when you had to debug a complex issue in a software application? What approach did you take, and what tools did you use to resolve it?', 'As you transition from HR to a technical role, what steps have you taken to enhance your Python skills and understanding of frameworks like Django or Flask?']


#### Find the last question in the collection

In [19]:
doc = collection.find(
    {}, 
    {"_id": 0, "questions": 1}
).sort("_id", -1).limit(1)

last_question_doc = list(doc)[0]
print(last_question_doc['questions'])


['Can you describe a specific lesson plan you developed for adult learners at Cambly, and how you tailored it to meet their individual needs? (Evaluating your ability to customize instruction for diverse learners.)', 'What strategies did you use to track student progress and provide feedback during your time at 51Talk, and how did you measure their success? (Assessing your methods for monitoring and enhancing student performance.)', 'In your experience as a Trial Lesson Teacher at Bubulearn, what challenges did you face in creating an engaging virtual classroom, and how did you overcome them? (Understanding your problem-solving skills and adaptability in a remote teaching environment.)', 'Can you share an example of how your communication and interpersonal skills helped you build rapport with adult learners in your previous roles? (Evaluating your soft skills and their impact on student engagement.)', 'How have you continued to develop your teaching skills or knowledge in English langu

In [20]:
query = [
    {
        "$sort": {
            "_id": -1  # Sort by _id descending to get the last document
        }
    },
    {
        "$project": {
            "_id": 0,
            "questions": 1
        }
    },
    {
        "$limit": 1
    }
]

result = list(collection.aggregate(query))
print(result[0]['questions'])


['Can you describe a specific lesson plan you developed for adult learners at Cambly, and how you tailored it to meet their individual needs? (Evaluating your ability to customize instruction for diverse learners.)', 'What strategies did you use to track student progress and provide feedback during your time at 51Talk, and how did you measure their success? (Assessing your methods for monitoring and enhancing student performance.)', 'In your experience as a Trial Lesson Teacher at Bubulearn, what challenges did you face in creating an engaging virtual classroom, and how did you overcome them? (Understanding your problem-solving skills and adaptability in a remote teaching environment.)', 'Can you share an example of how your communication and interpersonal skills helped you build rapport with adult learners in your previous roles? (Evaluating your soft skills and their impact on student engagement.)', 'How have you continued to develop your teaching skills or knowledge in English langu